In [2]:
from utils import *
import pandas as pd 
import numpy as np 
from statistics import mode

In [3]:
#Authentification  for Spotify API
sp, spt , spp = credentials()

#Extract the ids of User's Saved Tracks
track_ids = get_saved_song_ids(spt,sp)

#Extract the features of User's Saved Tracks
features = get_features_saved_songs(track_ids,sp)

In [ ]:
#In case you want to extract tracks from any  playlist
#track_ids_playlist = get_saved_song_ids(spt,sp,playlist_id = 'playlist_id', is_playlist=True)
#features = get_features_saved_songs(track_ids_playlist,sp)

In [5]:
#Save the results into a DataFrame
df = pd.DataFrame(features)
df

,id,name,album,artist
0,6Rkp6fEsvOTR7YxsW5uNyR,My Friend the Forest,Tripping with Nils Frahm,Nils Frahm
1,7kxnqBmbFilmSlYdDB6aBz,Moving Men - Bob Sinclar Remix,Moving Men (Bob Sinclar Remix),Myd
2,11ajcVj3qSyyMPUpTJUP3y,Protect The Land,Protect The Land / Genocidal Humanoidz,System Of A Down
3,1hM3r9AKtL3XZwPqRHcCMG,Entre Mis Brazos,Los Bunkers,Los Bunkers
4,05QhG4JOkDSPh4OsUCAUvM,Sentada Aqui En Mi Alma,Sincero,Chayanne
...,...,...,...,...
218,1KcOU0q9AMXV9xi4tSzzCc,Posthumous Forgiveness,The Slow Rush,Tame Impala
219,1s7fF5rM9oJmfpkK4OF8tu,Never Fade,Rainier Fog,Alice In Chains
220,5n5K6czwgPvZQpMTJVZ03O,Shock Me,Purple,Baroness
221,2Q7xYxV504kca2EYXOrMJx,Healer,Meanderthal,Torche


In [7]:
#Find the music genres from each song on the dataframe
df = finding_music_genre_on_spotify(df,sp)
df

Process Finished!


,id,name,album,artist,genre
0,6Rkp6fEsvOTR7YxsW5uNyR,My Friend the Forest,Tripping with Nils Frahm,Nils Frahm,"compositional ambient, electronica, neo-classi..."
1,7kxnqBmbFilmSlYdDB6aBz,Moving Men - Bob Sinclar Remix,Moving Men (Bob Sinclar Remix),Myd,"filter house, french indie pop, french indietr..."
2,11ajcVj3qSyyMPUpTJUP3y,Protect The Land,Protect The Land / Genocidal Humanoidz,System Of A Down,"alternative metal, nu metal, rap metal, rock"
3,1hM3r9AKtL3XZwPqRHcCMG,Entre Mis Brazos,Los Bunkers,Los Bunkers,"chilean rock, concepcion indie, latin alternat..."
4,05QhG4JOkDSPh4OsUCAUvM,Sentada Aqui En Mi Alma,Sincero,Chayanne,"latin, latin pop, puerto rican pop, tropical"
...,...,...,...,...,...
218,1KcOU0q9AMXV9xi4tSzzCc,Posthumous Forgiveness,The Slow Rush,Tame Impala,"australian psych, neo-psychedelic"
219,1s7fF5rM9oJmfpkK4OF8tu,Never Fade,Rainier Fog,Alice In Chains,"alternative metal, alternative rock, grunge, h..."
220,5n5K6czwgPvZQpMTJVZ03O,Shock Me,Purple,Baroness,"alternative metal, post-doom metal, progressiv..."
221,2Q7xYxV504kca2EYXOrMJx,Healer,Meanderthal,Torche,"miami metal, post-doom metal, post-hardcore, p..."


In [ ]:
#Use this function in case you don't find all the genres for tracks using the Spotify API
#df = find_music_genre_on_wikipedia(df)

In [12]:
#Save the dataframe for whatever kind of problem
df.to_csv('tracks.csv',index=False)

In [13]:
#Script to replace "null values" with  music genre of  the same artist or  same album in case exists on dataframe

artists = df.artist.unique().tolist()
albums = df.album.unique().tolist()
for album in albums:
    temp_df = df[df.album.isin([album])]
    for i in temp_df.index:
        if df.genre.iloc[i] == 'no information':
            df.genre.iloc[i] = mode(temp_df.genre.tolist())

for artist in artists:
    temp_df = df[df.artist.isin([artist])]
    for i in temp_df.index:
        if df.genre.iloc[i] == 'no information':
            df.genre.iloc[i] = mode(temp_df.genre.tolist())

In [23]:
#The ids of my target playlists (Playlists the I created on my Spotify User)
rock_playlist_id = '0Dc8VWYCL52AR3l816Yq0F'
metal_playlist_id = '7rvDjvhCXTS1bOdaDqY9BN'
pop_playlist_id = '7rff0sxjfg9cM4ShsnwOHW'
chill_playlist_id = '6259g0qo3cbJ1pINyfpP9b'

#The username id of your Spotify Account (Defined on utils.py script)
username = 'your_username_id'

"""function to add music to playlist by genre 

¡¡IMPORTANT!! after adding the tracks to the playlist, the track information of the   dataframe  will be delete to do not duplicate tracks for other playlists  

¡¡ALSO IMPORTANT!! you must to change to "make public" the preferences on your spotify playlist 
"""
def add_music_to_playlist_by_genre(df,username,genre,playlist_id,spp):

    df_ids = df['id'][df.genre.str.contains(genre)].tolist()
    df_index = df[df.genre.str.contains(genre)].index.tolist()
    df = df.drop(df_index,axis=0).reset_index(drop=True)


    #Add music to target playlist 
    spp.user_playlist_add_tracks(username,playlist_id=playlist_id,tracks=df_ids)

    print("Music Added to playlist!")

    return df 

In [20]:
#Adding Metalcore Tracks to Metal Playlist
df = add_music_to_playlist_by_genre(df,username,'metalcore',metal_playlist_id,spp)

Music Added to playlist!


In [28]:
#Adding Art Rock Tracks to Rock Playlist
df = add_music_to_playlist_by_genre(df,username,'art rock',rock_playlist_id,spp)

Music Added to playlist!


In [32]:
#Adding Ambient Tracks to Chill Playlist
df = add_music_to_playlist_by_genre(df,username,'ambient',chill_playlist_id,spp)

Music Added to playlist!
